In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import matplotlib.pyplot as plt 

In [2]:
data=pd.read_csv("E:\\github\\my upload\\nlp\\train_E6oV3lV.csv")

In [3]:
data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
data['label'].unique()

array([0, 1], dtype=int64)

In [5]:
data.shape

(31962, 3)

In [6]:
data=data.dropna()

In [7]:
import nltk
import re
from nltk.corpus import stopwords

In [8]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(data)):
    review = re.sub('[^a-zA-Z]', ' ', data['tweet'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [9]:
corpus

['user father dysfunct selfish drag kid dysfunct run',
 'user user thank lyft credit use caus offer wheelchair van pdx disapoint getthank',
 'bihday majesti',
 'model love u take u time ur',
 'factsguid societi motiv',
 'huge fan fare big talk leav chao pay disput get allshowandnogo',
 'user camp tomorrow user user user user user user user danni',
 'next school year year exam think school exam hate imagin actorslif revolutionschool girl',
 'love land allin cav champion cleveland clevelandcavali',
 'user user welcom gr',
 'ireland consum price index mom climb previou may blog silver gold forex',
 'selfish orlando standwithorlando pulseshoot orlandoshoot biggerproblem selfish heabreak valu love',
 'get see daddi today day gettingf',
 'user cnn call michigan middl school build wall chant tcot',
 'comment australia opkillingbay seashepherd helpcovedolphin thecov helpcovedolphin',
 'ouch junior angri got junior yugyoem omg',
 'thank paner thank posit',
 'retweet agre',
 'friday smile around

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [12]:
### Vocabulary size
voc_size=5000
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4810, 1408, 1938, 753, 4230, 1627, 1938, 4708],
 [4810, 4810, 817, 29, 1152, 4061, 4106, 3202, 2946, 4411, 2302, 2238, 2008],
 [506, 4862],
 [972, 3000, 99, 4749, 99, 3941, 937],
 [1589, 3833, 1484],
 [2663, 3331, 1764, 2697, 1269, 2768, 4169, 442, 3247, 4316, 1203],
 [4810, 539, 294, 4810, 4810, 4810, 4810, 4810, 4810, 4810, 3817],
 [1244, 802, 1450, 1450, 2845, 4009, 802, 2845, 3434, 864, 3709, 4443, 2812],
 [3000, 1276, 4248, 1100, 3043, 161, 4650],
 [4810, 4810, 4479, 2601],
 [1428, 2943, 3958, 2969, 3471, 1463, 4064, 4289, 913, 899, 918, 1490],
 [753, 2222, 1432, 4661, 3795, 1223, 753, 1850, 2222, 3000],
 [4316, 3801, 47, 1314, 4402, 4694],
 [4810, 570, 4801, 2792, 2853, 802, 4516, 2659, 41, 3955],
 [668, 1058, 4723, 3265, 2968, 4639, 2968],
 [476, 1449, 165, 4876, 1449, 2247, 400],
 [817, 2514, 817, 728],
 [953, 3921],
 [3651, 522, 3962, 4369, 3407, 4810, 4810, 3932, 3025, 651],
 [4554, 4053, 4296, 2595, 3258],
 [1868,
  651,
  2068,
  304,
  981,
  2553,
  2384,
  4454,
  2532

In [13]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1627 1938 4708]
 [   0    0    0 ... 2302 2238 2008]
 [   0    0    0 ...    0  506 4862]
 ...
 [   0    0    0 ... 3831 2429 2739]
 [   0    0    0 ... 3825 1624 4017]
 [   0    0    0 ...  817 4810 2064]]


In [14]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0, 4810, 1408, 1938,  753, 4230, 1627, 1938, 4708])

In [15]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
y=data['label']

In [18]:
len(embedded_docs),y.shape

(31962, (31962,))

In [19]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [20]:
X_final.shape,y_final.shape

((31962, 20), (31962,))

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [23]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 21414 samples, validate on 10548 samples
Epoch 1/10
21414/21414 [==============================] - 27s 1ms/sample - loss: 0.2102 - accuracy: 0.9359 - val_loss: 0.1435 - val_accuracy: 0.9507
Epoch 2/10
21414/21414 [==============================] - 16s 768us/sample - loss: 0.1034 - accuracy: 0.9623 - val_loss: 0.1466 - val_accuracy: 0.9500
Epoch 3/10
21414/21414 [==============================] - 17s 808us/sample - loss: 0.0757 - accuracy: 0.9728 - val_loss: 0.1523 - val_accuracy: 0.9519
Epoch 4/10
21414/21414 [==============================] - 18s 833us/sample - loss: 0.0581 - accuracy: 0.9800 - val_loss: 0.1745 - val_accuracy: 0.9496
Epoch 5/10
21414/21414 [==============================] - 18s 822us/sample - loss: 0.0423 - accuracy: 0.9862 - val_loss: 0.2168 - val_accuracy: 0.9466
Epoch 6/10
21414/21414 [==============================] - 18s 828us/sample - loss: 0.0310 - accuracy: 0.9894 - val_loss: 0.2383 - val_accuracy: 0.9471
Epoch 7/10
21414/21414 [======================

In [46]:
y_pred=model.predict_classes(X_test)

In [47]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[9609,  197],
       [ 360,  382]], dtype=int64)

In [54]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9471937808115283

Bidirectional

In [29]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 21414 samples, validate on 10548 samples
Epoch 1/10
21414/21414 [==============================] - 15s 718us/sample - loss: 0.1935 - accuracy: 0.9393 - val_loss: 0.1407 - val_accuracy: 0.9522
Epoch 2/10
21414/21414 [==============================] - 12s 540us/sample - loss: 0.0991 - accuracy: 0.9646 - val_loss: 0.1422 - val_accuracy: 0.9540
Epoch 3/10
21414/21414 [==============================] - 13s 615us/sample - loss: 0.0755 - accuracy: 0.9739 - val_loss: 0.1557 - val_accuracy: 0.9479
Epoch 4/10
21414/21414 [==============================] - 15s 723us/sample - loss: 0.0601 - accuracy: 0.9797 - val_loss: 0.1662 - val_accuracy: 0.9501
Epoch 5/10
21414/21414 [==============================] - 15s 683us/sample - loss: 0.0487 - accuracy: 0.9835 - val_loss: 0.2089 - val_accuracy: 0.9434
Epoch 6/10
21414/21414 [==============================] - 14s 652us/sample - loss: 0.0373 - accuracy: 0.9874 - val_loss: 0.2274 - val_accuracy: 0.9466
Epoch 7/10
21414/21414 [====================

In [48]:
y_pred1=model.predict_classes(X_test)

In [49]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred1)

array([[9609,  197],
       [ 360,  382]], dtype=int64)

In [55]:
accuracy_score(y_test,y_pred1)

0.9471937808115283

CNN

In [39]:
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import GlobalAveragePooling1D

In [44]:
embedding_vector_features=40
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model2.add(Conv1D(64, 4, activation='relu'))
model2.add(Dropout(0.3))
model2.add(GlobalAveragePooling1D())
model2.add(Dropout(0.3))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model2.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 17, 64)            10304     
_________________________________________________________________
dropout_6 (Dropout)          (None, 17, 64)            0         
_________________________________________________________________
global_average_pooling1d_2 ( (None, 64)                0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 210,369
Trainable params: 210,369
Non-trainable params: 0
________________________________________________

In [45]:
model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 21414 samples, validate on 10548 samples
Epoch 1/10
21414/21414 [==============================] - 3s 137us/sample - loss: 0.2528 - accuracy: 0.9300 - val_loss: 0.1605 - val_accuracy: 0.9433
Epoch 2/10
21414/21414 [==============================] - 2s 96us/sample - loss: 0.1198 - accuracy: 0.9589 - val_loss: 0.1448 - val_accuracy: 0.9529
Epoch 3/10
21414/21414 [==============================] - 2s 95us/sample - loss: 0.0857 - accuracy: 0.9714 - val_loss: 0.1566 - val_accuracy: 0.9508
Epoch 4/10
21414/21414 [==============================] - 2s 96us/sample - loss: 0.0714 - accuracy: 0.9763 - val_loss: 0.1732 - val_accuracy: 0.9518
Epoch 5/10
21414/21414 [==============================] - 2s 100us/sample - loss: 0.0586 - accuracy: 0.9813 - val_loss: 0.1873 - val_accuracy: 0.9478
Epoch 6/10
21414/21414 [==============================] - 2s 108us/sample - loss: 0.0500 - accuracy: 0.9832 - val_loss: 0.2077 - val_accuracy: 0.9498
Epoch 7/10
21414/21414 [=============================

In [50]:
y_pred2=model.predict_classes(X_test)

In [51]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred2)

array([[9609,  197],
       [ 360,  382]], dtype=int64)

In [56]:
accuracy_score(y_test,y_pred2)

0.9471937808115283